# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd # импорт библиотеки pandas

In [2]:
df = pd.read_csv('/datasets/data.csv') # чтения файла в переменную df


In [3]:
display(df.head(10)) # просмотр первых 10 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.info() # получение сводной информации о файле

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. Тип данных в пяти столбцах - object, в пяти - int64 и в двух - float64

Согласно документации к данным:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Количество значений в столбцах отличается. Значит в данных есть пропуски. Таких столбцов у нас два: трудовой стаж (days_employed) и тип занятости (total_income).


По первым 10 строкам уже видно, что в данных есть дубликаты. А в столбце стаж есть отрицательные и слишком большие цифры, что говорит о том, что данные нужно обработать и перепроверить. Возможно есть еще некорректные или неполные данные, но по первым 10 строкам их пока не видно.

**Вывод**

В каждой строке таблицы данные о заемщике. Часть данных рассказывает о самом заемщике, а часть о его ближайшем окружении (дети и семейное положение). 


Цель проекта: определить влияет ли ближайшее окружение клиента (наличие супруга и детей) на платежеспособность клиента.

На первый взгляд данных достаточно для проверки гипотез. Но встречаются проблемы в данных (пропуски и дубликаты).

Чтобы перейти непосредственно к проверке гипотез, нужно устранить проблемы в данных.

In [5]:
df.isna().mean()


children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

## Шаг 2. Предобработка данных

### Обработка пропусков

Посчитаем пропуски методом isna() в комбинации с методом sum(), чтобы получить количество пропусков, а не только значение True или False

In [6]:
df.isna().sum() # подсчет пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Есть предположение, что люди, которые никогда не работали просто пропускают графу со стажем при заполнении анкеты. В этом случае можно заменить данные на ноль. Но возможны и другие причины пропусков, например человек никогда не работал. Поскольку пропусков более 10% от общего количества, замена на ноль может сильно исказить данные. Поэтому лучше посчитать среднюю арифметическую, но для этого сначала необходимо избавится от отрицательных значений и слишком больших значений.

Избавлятся от отрицательных значений будем с помощью метода abs(), который возвращает абсолютное значение числа, т.е. его модуль.

In [7]:
df['days_employed'] = abs(df['days_employed']) # заменяем отрицательные значения на модуль
display(df.head(10)) # сразу проверяем, что данные изменились на примере первых 10 строк


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Как правило человек начинает работать не ранее 18 лет (но могут быть и исключения).А также стаж не может быть больше, чем возраст человека. Поэтому найдем все значения, которые больше, чем возраст клиента умноженный на 365 дней. Вероятно, что данные о стаже выражены в часах и тогда стаж разделим на 24 и получим данные в днях

Мы знаем, что в данном столбце есть пропуски. Чтобы цикл не прервал свою работу, при столкновении с пропущенным значением, используем конструкцию try-except. 

In [8]:
import warnings
warnings.simplefilter("ignore")

for i in range(len(df['days_employed'])):
    if df['days_employed'][i]>= df['dob_years'][i]*365:
        try:    
            df['days_employed'][i]/= 24 # код где может быть ошибка
        except:
            df['days_employed'][i]=df['days_employed'][i] # действия, если возникла ошибка
            

               


In [9]:
display(df.head(10)) # проверим первые 10 строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Данные изменились. В 5-ой строке было огромное значение, соответствующее более 900 лет работы, теперь данные приведены к дневному формату.

Стаж конечно не может быть больше возраста человека, но при этом в 20 лет можно иметь больше стажа работы, чем в 60. Поэтому считать средний стаж по возрасным группам не имеет смысла. Мы посчитаем среднее арифметическое по всей выборке и заменим пропущенные значения на него.

In [10]:
days_employed_mean = df['days_employed'].sort_values().mean() # отсортируем данные в столбце по возрастанию и посчитаем среднее арифметическое
display(days_employed_mean) # выведем значение на экран

4633.577338893336

С помощью метода fillna() заменяем пропуски. Он вместо NaN вставит в столбик тот аргумент, который мы ему укажем. В нашем случае - это среднее арифметическое.

In [11]:
df['days_employed'] = df['days_employed'].fillna(days_employed_mean) # заменяем пропуски на расчитанное среднее арифметическое

Данные о доходе в месяц (total_income) напрямую влияют на кредитоспособность людей. 

- необходимо заполнить пропуски в этом столбце

Поскольку банк не выдает кредитов людям без дохода, то заменять пропуски на ноль нельзя. Разброс дохода тоже может быть очень большим и брать среднее арифметическое не корректно. Для редактирования данных больше всего подойдет замена на медиану значения дохода. Но для начала нужно отсортировать сумму дохода по возрастанию.

Для сортировки используем метод sort_values(), который отсортирует наши данные в порядке возрастания. А затем с помощью метода fillna() заменим пропуски на медиану.

In [12]:
total_income_median = df['total_income'].sort_values().median() # отсортируем данные в столбце по возрастанию и посчитаем медиану

display(total_income_median) # выводим значение на экран

145017.93753253992

In [13]:
df['total_income'] = df['total_income'].fillna(total_income_median) # заменяем пропущенные значения в столбце на медиану

Проверим, что все пропуски заполнены

In [14]:
df.isna().sum() # подсчет пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

В процессе предобработки данных, обнаружилось, что в столбце возраст клиента (dob_years) есть нулевые значения (не пропущенные, а именно равные 0). Проверим:

In [15]:
display(df['dob_years'].min()) # определяем минимальное количество с помощью метода min()


0

Действительно нулевые значения есть. Не понятно только заменять их на среднее арифметическое или на медиану. Среднее арифметическое сложит все значения и поделит из на количество аргументов, а медиана находит значение находящееся в середине выборки. Посчитаем и то и другое.

In [16]:
display(df['dob_years'].mean()) # расчет среднего арифметического

43.29337979094077

In [17]:

display(df['dob_years'].sort_values().median()) # расчет медианы по отсортированному по возрастанию списку


42.0

Среднее арифметическое и медиана очень близки по значению, поэтому нет особой разницы, что использовать. Мы заменим ноль на медиану.

Для замены значения используем метод replace(). В качестве аргументов подадим ему старое значение (0) и новое значение (значение медианы)

In [18]:
dob_years_median = df['dob_years'].median() # сохраним в переменную значение медианы
df['dob_years'] = df['dob_years'].replace(0, dob_years_median) # заменим 0 на значение в переменной
display(df['dob_years'].min()) # проверим минимальное значение

19

Нули заменили и проверили, что теперь минимальное значение не равно 0

В анализе потребуется работа со столбцом, в котором указано количество детей. Проверим его на наличие "артефактов".

С помощью метода value_counts(), мы можем увидеть не только уникальные значения, но и частоту их появления.

In [19]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Опять обнаружены отрицательные значения. Возможно при заполнении анкеты клиенты ставили перед количеством дефис, но при загрузке в программу он попат туда, как минус. Заменим их на модуль числа с помощью метода abs()

In [20]:
df['children'] = abs(df['children']) # замена значения на его модуль

Также есть некоторое количество людей, у которых отмечено по 20 детей. По 20 детей написано даже у людей, которым до 30 лет. Вероятнее всего здесь опять некорректное введение данных. Возможно клиент указал 2.0, а система восприняла это, как 20.


In [21]:
children = df.loc[df['children'] == 20] # посмотрим строки с указанием количеста детей равное 20

display(children)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,90.066313,42,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,4633.577339,53,среднее,1,женат / замужем,0,M,компаньон,0,145017.937533,покупка жилой недвижимости
21404,20,494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


In [22]:
df['children'] = df['children'].replace(20, 2)

Мы заменили данные с помощью метода raplace() и вновь проверили данные с помощью value_counts()

In [23]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

На начальном этапе обнаружены следующие проблемы в данных:

- пропущены значения в столбцах с указанием трудового стажа и дохода в месяц
- обнаружены отрицательные значения в столбцах с количеством детей и трудовым стажем
- обнаружены большие числа в столбце трудовой стаж (данные указаны в часах, а не в днях)
- в столбце с указанием количеста детей обнаружены ненормально большое количество
- в столбце с указанием возраста некоторые значения равны 0.

Мы устранили указанные проблемы с помощью различных методов. Теперь пропущенные данные, отрицательные значения и просто некорректро введенные цифры не исказят результат.

### Замена типа данных

В столбцах с указанием трудового стажа в днях и дохода, после запятой слишком много цифр. В трудовом стаже - это будет считаться некорректным введением данных, так как трудовой стаж -это целое число. А в столбике с доходом, много символов после запятой усложняют восприятие.

Заменим вещественный тип танных на целочисленный в столбце с трудовым стажем (days_employed) методом astype() 

In [24]:
df['days_employed'] = df['days_employed'].astype('int') # приведение вещественного значения к целочисленному


Данные в столбце доход в месяц total_income, тоже приведем к целочисленному значению, поскольку округление до целого числа не сильно изменит суммарный доход даже при кредите на 10 лет. А вот читать в данных целые числа удобнее.

In [25]:
df['total_income'] = df['total_income'].astype('int') # приведение вещественного значения к целочисленному

In [26]:
df.info() # проверим информацию о данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Тип данных успешно обновлен.

**Вывод**

Поскольку трудовой стаж измеряется в целых днях, то данные в этом столбце заменили на целочисленные. Данные в столбце доход в месяц (total_income), тоже привели к целочислееному значению. После преобразований читать таблицу стало удобнее.

### Обработка дубликатов

Следующим этапом предобработки, будет проверка наличия дубликатов и их удаление.

Для начала проверим список уникальных значений в комбинации с их встречаемостью в таблице с помощью метода value_counts().

Последовательно проверим на наличие дубликатов столбики с указанием образования (education), семейного положения (family_status) и типа занятости (income_type).

Для начала проверим наличие полностью эдентичных строк методом df.duplicated().sum(). Если такие обнаружатся, то удалим дублирующиемя записи и обновим индексацию строк в таблице

In [27]:
df.duplicated().sum()

55

In [28]:
df = df.drop_duplicates().reset_index(drop=True)

In [29]:
df.duplicated().sum()

0

Начнем с образования:

In [30]:
df['education'].value_counts() # поиск уникальных значений с указанием их встречаемости 

среднее                13704
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Сразу в глаза бросились одни и те же наименования, но написанные разным шрифтом. 

Приведем значения к нижнему регистру с помощью метода str.lower()

In [31]:
df['education'] = df['education'].str.lower() # приведение значений к нижнему регистру
df['education'].value_counts() # проверим список уникальных значений

среднее                15187
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Список сильно сократился. Дубликаты удалены. Переходим к сдедующему столбцу:

In [32]:
df['family_status'].value_counts() # поиск уникальных значений с указанием их встречаемости

женат / замужем          12344
гражданский брак          4162
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Явных дубликатов нет, но строка "Не женат / не замужем" содержит прописные буквы. Если в таблицу добавят новые данные, но уже с этой строкой, написанной строчными буквами- образуется дубликат. Чтобы этого избежать в будущем, сразу приведем все символы к нижнему регистру, методом str.lower()

In [33]:
df['family_status'] = df['family_status'].str.lower() # приведение значений к нижнему регистру

Столбик с семейным статусом отработали, перейдем к типом занятости:

In [34]:
df['income_type'].value_counts() # поиск уникальных значений с указанием их встречаемости 

сотрудник          11090
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

В данном столбце дубликатов не обнаружено.

**Вывод**

В данной части анализа мы удаляли дубликаты. Наличие дубликатов при группировке и приводит к неверному трактованию результатов. Можно было бы еще больше объединить значения, но для этого нужна дополнительная информация от банка, например является ли "не женат/не замужем", "в разводе" "и вдовец / вдова" для них единым признаком заемщика. Но без дополнительной информации, мы пока ничего объединять не будем.

### Лемматизация

В столбике с указание цели кредита (purpose) значения произвольные, т.е например одна и та же цель может быть описана разными словами. 

С помощью метода unique() посмотрим уникальные значения в столбце. Поскольку подсчет встречаемости, нам сейчас не важен, то использовали именно этот метод, а не value_counts()

In [35]:
df['purpose'].unique() # выделение уникальных значений

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Для того, чтобы удалить дубликаты из такого списка проводят лемматизацию. Лемматизация - это преобразование слов в лемму, то есть в их первоначальную словарную форму. После выделения лемм, можно будет обработать дубликаты.

Выделим леммы в значениях столбца и составим из них список. 

    

Поскольку лемматизация работает с одной строкой, напишем цикл, в котором лемматизация последовательно будет проведена по всем строкам, а результат лемматизации сохранен в общий список. Создадим пустой список purpose_lemm и будем добавлять в него значения методом append(). 

In [36]:
from pymystem3 import Mystem # подключение лемматизатора для слов на русском языке
m = Mystem()

purpose_lemm = [] # пустой список, куда будет сохранен итоговый список результатов всех лемматизации
for i in range(len(df['purpose'].unique())): # цикл с указаием по каким позициям будет проводится лемматизация
        
    lemmas = ' '.join(m.lemmatize(df['purpose'][i])) # выделение леммы по строке и объединение в список
    from collections import Counter
    purpose_lemm.append(lemmas) 
    
display(purpose_lemm)

['покупка   жилье \n',
 'приобретение   автомобиль \n',
 'покупка   жилье \n',
 'дополнительный   образование \n',
 'сыграть   свадьба \n',
 'покупка   жилье \n',
 'операция   с   жилье \n',
 'образование \n',
 'на   проведение   свадьба \n',
 'покупка   жилье   для   семья \n',
 'покупка   недвижимость \n',
 'покупка   коммерческий   недвижимость \n',
 'сыграть   свадьба \n',
 'приобретение   автомобиль \n',
 'покупка   жилой   недвижимость \n',
 'строительство   собственный   недвижимость \n',
 'недвижимость \n',
 'строительство   недвижимость \n',
 'на   покупка   подержать   автомобиль \n',
 'на   покупка   свой   автомобиль \n',
 'недвижимость \n',
 'приобретение   автомобиль \n',
 'на   покупка   подержать   автомобиль \n',
 'сыграть   свадьба \n',
 'операция   с   коммерческий   недвижимость \n',
 'покупка   недвижимость \n',
 'образование \n',
 'строительство   собственный   недвижимость \n',
 'строительство   собственный   недвижимость \n',
 'строительство   жилой   недвижимос

Напишем функцию, которая будет возвращать результат лемматизации в новый столбец purpose_lemm. Метод apply() записывает в новый столбец значения возвращенные функцией.

In [37]:
def purpose_change(purpose):
    lemmas_row = m.lemmatize(purpose)
    if 'автомобиль' in lemmas_row:
        return 'автомобиль'
    if 'свадьба' in lemmas_row:
        return 'свадьба'
    if 'образование' in lemmas_row:
        return 'образование'
    if 'свадьба' in lemmas_row:
        return 'свадьба'
    if ['жилье' in lemmas_row and 'недвижимость' in lemmas_row]:               
        return 'недвижимость'
    
    
    
df['purpose_lemm'] = df['purpose'].apply(purpose_change)
display(df.head(10))


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


Проверяем уникальные значения:

In [38]:
df['purpose_lemm'].unique()


array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

**Вывод**

Лемматизация позволила ним удалить дубликаты в столбике с целью кредита (purpose). Обработанные данные мы сохранили в отдельный столбец. Вместо достаточно емкого списка, мы получили 4 значения,  которые легко читаются и все группировки уже будем проводить по нему. 

### Категоризация данных

В нашей таблице есть категории в виде названий. У них разная длина и это приводит к усложнению восприятия информации.

Для оптимизации дальнейших подсчетов, создадим отдельные файлы-словари, где названию категории соответствует номер.

Сначала создадим файл-словарь для семейного положения:

В переменную family_status_id добавим два столбца: данные по идентификатору семейного положения и по названию семейного положения. А затем удалим дубликаты с обновление индексации и данные перезапишем в переменную family_status_id.

In [39]:
family_status_id = df[['family_status_id', 'family_status']] # выделение двух столбцов из датасета df 

In [40]:
family_status_id = family_status_id.drop_duplicates().reset_index(drop = True) # удаление дубликатов с обновлением индексации
display(family_status_id) # просмотр результата

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


К столбцу с лемматизированной целью кредита нет идентификаторов.

Создадим словарь, в котором название категории цели кредита будет ключом, а справа укажем значение- номер категории.

In [41]:
purpose_id ={'недвижимость': 0, 'автомобиль':1, 'образование':2, 'свадьба':3} # создание словаря

С помощью метода map() добавим номер идентификатора цели кредита в датасет df

In [42]:
df['purpose_lemm_id'] = df['purpose_lemm'].map(purpose_id) 
    


Затем проведем операцию аналогичную той, что мы сделали, когда категоризировали семейное положение.

In [43]:
purpose_lemm_id = df[['purpose_lemm_id', 'purpose_lemm']] # выделение двух столбцов, для создания файла-ключа

In [44]:
purpose_lemm_id = purpose_lemm_id.drop_duplicates().reset_index(drop = True) # удаление дубликатов
display(purpose_lemm_id)

,purpose_lemm_id,purpose_lemm
0,0,недвижимость
1,1,автомобиль
2,2,образование
3,3,свадьба


У нас еще есть один столбик по которому нужно провести категоризацию. Если этого не сделать, про при анализе придется смотреть каждую строчку таблицы, ведь средний доход у всех разный и если и есть дубликаты, то их очень мало.

Сначала посмотрим с какими минимальными и максимальными значениями нам придется столкнутся в столбце с указанием среднего дохода.

In [45]:
display(df['total_income'].min()) # расчет минимального значения
display(df['total_income'].max()) # расчет максимального значения


20667

2265604

Минимально 20667, а максимум 226504. Поставим первую группу до 50000, потом два увеличения на 100000, затем на 150000, а после дохода в 500000, укрупняем и добавляем по 500000

Для начала напишем функцию, которая отнесет разный уровень дохода в разные группы и присвоит им номер.

In [46]:
def total_income_id(total_income): # функция для отнесения дохода в определенную группу и присвоения ей номера
        
    if total_income <= 50000:
        return 0
    if total_income <= 150000:
        return 1
    if total_income <= 250000:
        return 2
    if total_income <= 350000:
        return 3
    if total_income <= 500000:
        return 4
    if total_income <= 1000000:
        return 5
    if total_income <= 1500000:
        return 6
    if total_income <= 2000000:
        return 7
     
    return 8 

Аналогично, как и с целью кредита с помощью метода apply() заносим в новый столбец таблицы идентификатор для уровня дохода.

In [47]:
df['total_income_id']=df['total_income'].apply(total_income_id) # добавление столбца со значениями возвращенными функцией


Создаем словарь, где ключ - это номер группы, а значение - это диапазон значений дохода 

In [48]:
total_income_dict = {0: 'доход до 50 000 руб.', 1: 'доход от 50 001 до 150 000', 2: 'доход от 150 001 до 250 000', 3: 'доход от 250 001 до 350 000', 4:'доход от 300 001 до 500 000', 5: 'доход от 500 001- до 1 000 000', 6: '1 000 001-1 500 000', 7: 'доход от 1 500 001 до 2 000 000', 8: 'доход более 2 000 000'}

С помощью метода map() добавим номер идентификатора дохода в датасет df

In [49]:
df['total_income_group'] = df['total_income_id'].map(total_income_dict)

In [50]:
total_income_group = df[['total_income_id','total_income_group']] # выделение двух столбцов, для создания файла-ключа

In [51]:
total_income_group = total_income_group.drop_duplicates().reset_index(drop = True) # удаление дубликатов
display(total_income_group)

,total_income_id,total_income_group
0,3,доход от 250 001 до 350 000
1,1,доход от 50 001 до 150 000
2,2,доход от 150 001 до 250 000
3,5,доход от 500 001- до 1 000 000
4,4,доход от 300 001 до 500 000
5,0,доход до 50 000 руб.
6,6,1 000 001-1 500 000
7,7,доход от 1 500 001 до 2 000 000
8,8,доход более 2 000 000


**Вывод**

Мы провели категоризацию данных. Теперь при анализе мы можем группировать необходимые данные привязывая к выделенным категориям. Также мы облегчили визуальную работу с таблицей. Для дальнейшей фильтрации нам не нужно будет набирать полное название категории или как для дохода пробегаться по всем строкам таблицы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для определения зависимости между наличием детей и вероятностью возврата кредита, проведем группировку столбца наличия задолженности (debt) по столбцу количество детей. И сразу посчитаем сумму невозвращенных кредитов и общую сумму выданных кредитов по каждой группе. Чтобы одновременно посчитать и сумму и общее количество значений используем метод agg().

In [52]:
children_dert_final = df.groupby('children').agg({'debt': ['sum', 'count']}) # группировка столбца наличия задолженности (debt) по столбцу количество детей и расчет суммы невозвращенных кредитов и общего количества кредитов


Посчитаем конверсию по каждой группе и сохраним ее в столбик convers_percents (конверсия указана в процентах):

In [53]:
children_dert_final['convers_percents'] = (df.groupby('children')['debt'].sum()/df.groupby('children')['debt'].count())*100

In [54]:
display(children_dert_final)

debt        convers_percents
           sum  count                 
children                              
0         1063  14106         7.535800
1          445   4856         9.163921
2          202   2128         9.492481
3           27    330         8.181818
4            4     41         9.756098
5            0      9         0.000000

**Вывод**

На первый взгляд количество невозвращенных кредитов в группе бездетных клиентов больше, но при этом и количество самих кредитов выше. В группе где один ребенок у заемщика, процент невозвратов больше, а у клиентов с двумя детьми еще больше. У клиентов с тремя детьми, процент невозвратов снижается, по сравнению с предыдущей группой, но возможно, что какие-то некорректные данные остались вне поля нашего зрения. По столь малой выборке, которая приходится на группу с тремя детьми, сложно сделать однозначный вывод. По группе с четырьмя детьми процент невозвратов растет, по сравнению в другими группами, но опять же выборка слишком мала, для выводов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для определения зависимости между семейным положением и вероятностью возврата кредита, проведем группировку столбца наличия задолженности (debt) по столбцу идентификатора семейного статуса. И сразу посчитаем сумму невозвращенных кредитов и общую сумму выданных кредитов по каждой группе. 

С помощью метода pivot_table() сделаем сводную таблицу в которой посчитаем сколько невыплаченных кредитов приходится на каждую группу заемщиков.

In [55]:
family_status_final =df.pivot_table(index='family_status_id', columns='family_status', values='debt', aggfunc='sum')
display(family_status_final)

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
family_status_id,,,,,
0,NaN,NaN,NaN,931.0,NaN
1,NaN,NaN,388.0,NaN,NaN
2,NaN,63.0,NaN,NaN,NaN
3,85.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,274.0


Расчитаем конверсию:

In [56]:
family_status_final['convers_percents'] = (df.groupby('family_status_id')['debt'].sum()/df.groupby('family_status_id')['debt'].count())*100 # расчет конверсии

In [57]:
display(family_status_final)

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем,convers_percents
family_status_id,,,,,,
0,NaN,NaN,NaN,931.0,NaN,7.542126
1,NaN,NaN,388.0,NaN,NaN,9.322441
2,NaN,63.0,NaN,NaN,NaN,6.569343
3,85.0,NaN,NaN,NaN,NaN,7.112971
4,NaN,NaN,NaN,NaN,274.0,9.750890


**Вывод**

Самое большое количество кредитов в группе женатых людей и конверсия у них 7,5%. Достаточно высокая конверсия у людей не женатых и находящихся в граждансом браке. Можно однозначно сказать, что процент невозврата по клиентам не состоящим в зарегистрированном браке выше. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для определения зависимости между уровнем дохода и вероятностью возврата кредита, проведем группировку столбца наличия задолженности (debt) по столбцу идентификатора дохода. И сразу посчитаем сумму невозвращенных кредитов и общую сумму выданных кредитов по каждой группе. Чтобы одновременно посчитать и сумму и общее количество значений используем метод agg()

In [58]:
total_income_id = df.groupby('total_income_group').agg({'debt': ['sum', 'count']})

Для этого расчета у нас также
есть категоризированные данные. Применим их и с помощью метода merge() соединим две таблицы: категоризированные данные и сгруппированные данные по кредиту.

In [59]:
total_income_final = total_income_group.merge(total_income_id, on='total_income_group', how='left')

Расчитаем конверсию:

In [60]:
total_income_final['convers_percents'] = (df.groupby('total_income_id')['debt'].sum()/df.groupby('total_income_id')['debt'].count())*100

In [61]:
display(total_income_final)

,total_income_id,total_income_group,"(debt, sum)","(debt, count)",convers_percents
0,3,доход от 250 001 до 350 000,139,1954,6.182796
1,1,доход от 50 001 до 150 000,992,11914,8.326339
2,2,доход от 150 001 до 250 000,532,6372,8.349027
3,5,доход от 500 001- до 1 000 000,12,197,7.113613
4,4,доход от 300 001 до 500 000,41,636,6.446541
5,0,доход до 50 000 руб.,23,372,6.091371
6,6,1 000 001-1 500 000,1,18,5.555556
7,7,доход от 1 500 001 до 2 000 000,0,5,0.000000
8,8,доход более 2 000 000,1,2,50.000000


**Вывод**

Как оказалось, люди с доходом до 50000 более дисциплинированные в плане погашения долгов, чем люди с большим доходом. Категории с доходом более 500000, мы рассмтривать не будем, потому что количество в выборке по этим группам очень маленькое. Хуже всего возвращают кредиты люли из групп с доходом от 50000 до 250000.  

- Как разные цели кредита влияют на его возврат в срок?

Для определения зависимости между целью кредита и вероятностью возврата кредита, проведем группировку столбца наличия задолженности (debt) по столбцу идентификатора цели кредита. И сразу посчитаем сумму невозвращенных кредитов и общую сумму выданных кредитов по каждой группе. Чтобы одновременно посчитать и сумму и общее количество значений используем метод agg()

In [62]:
purpose_group = df.groupby('purpose_lemm_id').agg({'debt': ['sum', 'count']})

Применим категоризированные данные и с помощью метода merge() соединим две таблицы: категоризированные данные и сгруппированные данные по кредиту.

In [63]:
purpose_final = purpose_lemm_id.merge(purpose_group, on='purpose_lemm_id', how='left')


Расчитаем конверсию:


In [64]:
purpose_final['convers_percents'] = (df.groupby('purpose_lemm_id')['debt'].sum()/df.groupby('purpose_lemm_id')['debt'].count())*100

In [65]:
display(purpose_final)

,purpose_lemm_id,purpose_lemm,"(debt, sum)","(debt, count)",convers_percents
0,0,недвижимость,782,10814,7.231367
1,1,автомобиль,403,4308,9.354689
2,2,образование,370,4014,9.217738
3,3,свадьба,186,2334,7.969152


**Вывод**

На покупку недвижимости выдано больше всего кредитов, но невозврат в этой группе меньше, чем на автомобиль. Также высокий процент показывает конверсия кредита на обучение.

## Шаг 4. Общий вывод

В процессе анализа данных была проведена предобработка данных, включающая:
- удаление пропусков
- замену некорректных значений
- изменены типы данных в столбцах с указанием дохода и стажа
- удаление дубликатов с помощью приведения значений к нижнему регистру
- использована лемматизация для удаления дубликатов по столбцу с укащзанием цели кредита в произвольной форме
- проведена категоризация данных.


Полученные данные говорят о том, что уровень дохода, цель кредита, семейное положение и наличие детей, каждый в своем роде влияет на возврат кредита. Но для более точной оценки нужно оценивать эти показатели вместе, а не отдельно. За основу такого анализа я бы взяла уровень дохода и изучила бы более подробно группу с доходом от 50000 до 250000. Именно в этой группе посмотрела как влияет семейное положение, количество детей и цель кредита на возврат заемных средств.